## An Electrode Design Optimisation Example

A design optimisation example loosely based on work by L.D. Couto available at [[1]](https://doi.org/10.1016/j.energy.2022.125966).

The target is to maximise the gravimetric energy density over a range of possible design parameter values, including for example:

cross-sectional area = height x width (only need change one), electrode widths, particle radii, volume fractions and separator width.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
import numpy as np
import pybamm
from pybamm import Parameter

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

## Optimising the parameters

First, we define the model and parameter values to be used for the parameter optimisation.

In [ ]:
model = pybamm.lithium_ion.SPMe()
pybop.pybamm.add_variable_to_model(model, "Gravimetric energy density [W.h.kg-1]")

parameter_values = pybamm.ParameterValues("Chen2020")
pybop.pybamm.set_formation_concentrations(parameter_values)
parameter_values.update(
    {
        "Electrolyte density [kg.m-3]": Parameter("Separator density [kg.m-3]"),
        "Negative electrode active material density [kg.m-3]": Parameter(
            "Negative electrode density [kg.m-3]"
        ),
        "Negative electrode carbon-binder density [kg.m-3]": Parameter(
            "Negative electrode density [kg.m-3]"
        ),
        "Positive electrode active material density [kg.m-3]": Parameter(
            "Positive electrode density [kg.m-3]"
        ),
        "Positive electrode carbon-binder density [kg.m-3]": Parameter(
            "Positive electrode density [kg.m-3]"
        ),
        "Cell mass [kg]": pybop.pybamm.cell_mass(),
    },
    check_already_exists=False,
)

Next, we define the model parameters for optimisation. Furthermore, PyBOP provides functionality to define a distribution for the parameters. The initial parameters values used in the optimisation will be randomly drawn from the prior distribution.

In [ ]:
parameter_values.update(
    {
        "Positive electrode thickness [m]": pybop.ParameterDistribution(
            pybop.Gaussian(7.56e-05, 0.05e-05, truncated_at=[65e-06, 10e-05]),
        ),
        "Positive particle radius [m]": pybop.ParameterDistribution(
            pybop.Gaussian(5.22e-06, 0.05e-06, truncated_at=[2e-06, 9e-06]),
        ),
    }
)

Next, we construct the experiment for design optimisation and the initial state-of-charge,

In [ ]:
experiment = pybamm.Experiment(["Discharge at 1C until 2.5 V (5 seconds period)"])
initial_soc = 0.7

We can now define the simulator, cost function, which in this example is based on the gravimetric energy density of the cell, and construct the problem.

In [ ]:
simulator = pybop.pybamm.Simulator(
    model,
    parameter_values=parameter_values,
    protocol=experiment,
    initial_state={"Initial SoC": initial_soc},
    solver=pybamm.CasadiSolver(),
)
cost = pybop.DesignCost(target="Gravimetric energy density [W.h.kg-1]")
problem = pybop.Problem(simulator, cost)

Next we construct an optimiser. This class provides the algorithm to optimise the parameters. For this example, we use particle swarm optimisation (PSO). Due to the computational requirements of the design optimisation methods, we limit the number of iterations for this example.

In [ ]:
options = pybop.PintsOptions(verbose=True, max_iterations=15)
optim = pybop.PSO(problem, options=options)

Finally, we run the optimisation and return the values obtained,

In [ ]:
result = optim.run()
print(f"Initial gravimetric energy density: {result.cost[0]:.2f} W.h.kg-1")
print(f"Optimised gravimetric energy density: {result.best_cost:.2f} W.h.kg-1")

| Iter: 1 | Evals: 6| Best Parameters: [8.32936135e-05 5.34874257e-06] | Best Cost: 272.4725741332926


| Iter: 2 | Evals: 12| Best Parameters: [8.32936135e-05 5.34874257e-06] | Best Cost: 272.4725741332926


| Iter: 3 | Evals: 18| Best Parameters: [8.46531639e-05 3.79767350e-06] | Best Cost: 278.4671427205715


| Iter: 4 | Evals: 24| Best Parameters: [8.46531639e-05 3.79767350e-06] | Best Cost: 278.4671427205715


| Iter: 5 | Evals: 29| Best Parameters: [8.46531639e-05 3.79767350e-06] | Best Cost: 278.4671427205715


...

| Iter: 7 | Evals: 37| Best Parameters: [8.55293313e-05 3.53706776e-06] | Best Cost: 278.642235228317


| Iter: 8 | Evals: 43| Best Parameters: [8.55293313e-05 3.53706776e-06] | Best Cost: 278.642235228317


| Iter: 9 | Evals: 49| Best Parameters: [8.55293313e-05 3.53706776e-06] | Best Cost: 278.642235228317


| Iter: 10 | Evals: 55| Best Parameters: [8.29860246e-05 2.58546828e-06] | Best Cost: 287.01440153288445


OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [7.56456024e-05 5.27456414e-06]
  Optimised parameters: [8.38277729e-05 2.13404650e-06]
  Best cost: 288.74937116651336
  Optimisation time: 65.29159212112427 seconds
  Number of iterations: 15
  Number of evaluations: 77
  Reason for stopping: Maximum number of iterations (15) reached.
Initial gravimetric energy density: 254.67 W.h.kg-1
Optimised gravimetric energy density: 288.75 W.h.kg-1


## Plotting and visualisation

PyBOP provides various plotting utilities to visualise the results of the optimisation.

### Cost landscape

We can visualise the cost landscape and the path taken by the optimiser:

In [ ]:
result.plot_surface();

### Comparing system response

We can also plot the system's response using the estimated parameters compared to the initial parameters:


In [ ]:
problem.target = "Voltage [V]"
pybop.plot.problem(problem, inputs=result.best_inputs, title="Optimised Comparison");